In [1]:
!pip install pandas_profiling==2.9.0

In [2]:
from azureml.widgets import RunDetails
from pandas_profiling import ProfileReport
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset

In [3]:
# Get current workspace from config
ws = Workspace.from_config()
    
ws.get_details()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: OptimizePipeline
Azure region: eastus2
Subscription id: c04b3d3f-4994-454d-96ff-aa3f2050b57f
Resource group: testingMLFunctionnalities


In [4]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Covid19InfectionsDataset"
description_text = "Covid19 Vaccination DataSet from Github"
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./github/owid-covid-data.csv'],
                       target_path ='train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)

if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    original_path = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
    ds = TabularDatasetFactory.from_delimited_files(original_path, infer_column_types=True, separator=',', header=True, index=False)
    
    #Register Dataset in Workspace
    dataset = ds.register(workspace=ws,
                          name=key,
                          description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

Uploading an estimated of 1 files
Uploading ./github/owid-covid-data.csv
Uploaded ./github/owid-covid-data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
count,7.590800e+04,75906.000000,74905.000000,6.656900e+04,66727.000000,74905.000000,75495.000000,75493.000000,74497.000000,66169.000000,...,70027.000000,69223.000000,69633.000000,70285.000000,70889.000000,71817.000000,35587.000000,64836.000000,73433.000000,70831.000000
mean,6.933484e+05,5267.220799,5262.048175,2.021219e+04,132.896039,117.110210,8290.103995,68.937740,68.699809,192.249212,...,30.544706,8.788616,5.569119,19140.773439,257.327124,7.803056,50.978244,3.033377,73.152441,0.727647
std,4.850087e+06,32675.802795,32254.835800,1.186788e+05,725.870682,663.331248,16214.400983,171.161501,144.339264,340.564356,...,9.114126,6.233355,4.256960,19762.848508,118.590918,3.948874,31.766562,2.466047,7.550791,0.150082
min,1.000000e+00,-74347.000000,-6223.000000,1.000000e+00,-1918.000000,-232.143000,0.001000,-2153.437000,-276.825000,0.001000,...,15.100000,1.144000,0.526000,661.240000,79.370000,0.990000,1.188000,0.100000,53.280000,0.394000
25%,7.620000e+02,2.000000,6.143000,4.000000e+01,0.000000,0.000000,161.791000,0.151000,1.040000,5.903000,...,22.200000,3.441000,2.043000,4466.507000,167.295000,5.290000,20.859000,1.300000,67.880000,0.602000
50%,8.610000e+03,62.000000,75.286000,2.650000e+02,2.000000,1.143000,1137.508000,6.710000,8.864000,33.396000,...,29.700000,6.378000,3.871000,12951.839000,242.648000,7.110000,49.839000,2.400000,74.530000,0.748000
75%,9.238175e+04,695.000000,721.000000,2.567000e+03,16.000000,12.571000,7797.146000,59.754000,67.241000,204.623000,...,39.100000,14.312000,8.678000,27216.445000,329.635000,10.080000,83.241000,4.000000,78.730000,0.848000
max,1.254917e+08,880902.000000,739564.429000,2.755210e+06,17903.000000,14431.429000,151258.655000,8652.658000,2648.773000,2416.171000,...,48.200000,27.049000,18.493000,116935.600000,724.417000,30.530000,98.999000,13.800000,86.750000,0.957000


In [ ]:
profile = ProfileReport(df, title="Raw Covi19 Data Profiling", explorative=True)
profile.to_notebook_iframe()
profile.to_file("outputs/DataCleaningReport.html")

Summarize dataset:   0%|          | 0/73 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]